Normalization

In [103]:
import numpy as np
import pandas as pd
import torch
import os
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import urllib.request
import seaborn as sns

np.random.seed(13)

pd.set_option('display.max_columns', None)

# DATA ACQUISITION

In [104]:
datasets_csv = ["movies.csv", "ratings.csv", "genome-scores.csv", "genome-tags.csv", "links.csv", "tags.csv"]
datasets_name = [i[:-4].replace("-", "_") for i in datasets_csv]

root_URL = "http://github.com/MickPerl/DataAnalyticsProject/releases/download/datasets/"
remote_url = ["".join([root_URL, i]) for i in datasets_csv]

In [105]:
try :
    if os.getcwd().split("/")[-1] != 'ml-25m':
        os.chdir('data/ml-25m/')
except FileNotFoundError :
    os.makedirs("data/ml-25m")
    os.chdir('data/ml-25m/')

In [ ]:
# per vedere se i csv sono già presenti
!ls

# per assicurarsi di essere nel virtual environment
!which python

In [107]:
for i in range(len(datasets_csv)):
    try :
        globals()["_".join(['df', datasets_name[i]])] = pd.read_csv(datasets_csv[i])
    except FileNotFoundError:
        print(f"Download in progress of {datasets_csv[i]}")
        file = urllib.request.urlretrieve(remote_url[i], datasets_csv[i])
        globals()["_".join(['df', datasets_name[i]])] = pd.read_csv(file[0])

Download in progress of genome-scores.csv
Download in progress of genome-tags.csv
Download in progress of links.csv
Download in progress of tags.csv


# PRE PROCESSING

In [ ]:
df_movies["year"] = [i.strip()[-5:-1] for i in df_movies["title"]]
df_movies["title_length"] = [len(i) for i in df_movies["title"]]
df_movies.drop("title", inplace=True, axis=1)

In [ ]:
# SPIEGARE
mlb = MultiLabelBinarizer()
df_movies = df_movies.join(pd.DataFrame(
                mlb.fit_transform(df_movies.pop('genres').str.split('|')),
                index=df_movies.index,
                columns=mlb.classes_))

In [ ]:
df_movies['genres'] = df_movies.iloc[:,3:23].values.tolist()
df_movies.head()

Hamming

In [ ]:
np.sum(np.bitwise_xor([1,1,0,0,0],[1,0,1,1,0]))

In [ ]:
df_genome = pd.merge(df_genome_tags, df_genome_scores, on="tagId")
df_genome = df_genome.pivot(index='movieId', columns='tag', values="relevance")
df = pd.merge(df_movies, df_genome, on="movieId")

In [ ]:
df_ratings = df_ratings.iloc[:,[1,2]]
df_ratings = df_ratings.groupby(by='movieId').mean()
df = pd.merge(df_ratings, df, on="movieId")

In [ ]:
df_X = df.loc[:,df.columns != 'rating']
df_Y = df['rating']

X_train, X_test, Y_train, Y_test = train_test_split(df_X, df_Y, test_size=0.2, random_state=20)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=20)

In [ ]:
pd.set_option('display.max_rows', df.shape[0]+1)
df.describe()

Select rows with missing data

In [ ]:
len(df[pd.isnull(df).any(axis=1)])
# df.fillna(interesting_metric)     or      df.dropna()

DupChecking correlatio

In [ ]:
any(df.duplicated())

Standardizzazione

In [ ]:
# NORMALIZATION
Y_train = (Y_train - Y_train.mean()) / Y_train.std()
Y_train.plot(kind ='density')
plt.show()

In [ ]:
pd.set_option('expand_frame_repr', False)
list(df.columns)

In [ ]:
# PRE DATA VISUALIZATION
df_.plot(kind ='density')
plt.show()

## Checking correlation

In [ ]:
sns.set_theme(style="ticks")
sns.pairplot(df, hue="species")

In [ ]:
# PRE DATA VISUALIZATION
df_Y.plot(kind ='density')
plt.show()